In [ ]:
# Program after checking siblings in Aruba machine remotely
scrubber_asn = "32787" # Scrubber asn
year = "2022"
day = "01" # It is default day for our analysis

import pandas as pd
import csv
from concurrent.futures import ProcessPoolExecutor
import os
import re

path = "/home/shyam/jupy/ddos_scrubber/data/as"+scrubber_asn+"/"+year+"/"

In [ ]:
print("Downloading files from Aruba machine.\n")
os.system("scp aruba-shyam:/home/shyam/data/workspace/project/notebooks/unique_optimized_provider_not_as*_v3.csv "+path)
print("Downloaded.")

In [ ]:
print("Step 6: Finding customers that have sibling ASes.\n")
pattern = r"unique_optimized_provider_not_as"+scrubber_asn+".*\_v3.csv$" 
for filename in os.listdir(path):
     # Read only processed files
    if re.search(pattern, filename):
        day = filename.split('_')[5] # Get day from file name
        mon = filename.split('_')[6] # Get month name from file name
        year = filename.split('_')[7]
        
        df = pd.read_csv(path + filename)
        # Confirmed customers (AS198949 comes immediately after the siblings) after sibling check
        confirmed_customers_after_sibling_check = df.loc[(df['siblings'] == 1) & (df['new_provider_sibling_check'] == int(scrubber_asn))]
#         print("%s number of records have sibling ASes and contain AS%s as a provider (not a second last hop in AS path though)" %(len(confirmed_customers_after_sibling_check), scrubber_asn))

        no_sibling_different_provider_records = df.loc[(df['siblings'] == 0) & (df['new_provider_sibling_check'] != int(scrubber_asn))]
#         print("%s number of records do not have sibling ASes and contain another provider" %len(no_sibling_different_provider_records))
        no_sibling_different_provider_records
print("Step 6 completed.")

In [ ]:
print("Step 7: Finding direct customers, confirmed_customers_after_sibling_check and path prepending.\n")
path = "/home/shyam/jupy/ddos_scrubber/data/as"+scrubber_asn+"/"+year+"/"

months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
all_record_ases = [] # Stores number of customer ases for a month
all_record_prefixes = [] # Stores number of customer prefixes for a month


for mon in months:
    single_record_ases = [] # Stores number of customer ases for a month
    single_record_prefixes = [] # Stores number of customer prefixes for a month

    confirmed_customers1 = pd.read_csv(path + "confirmed_customers_as"+scrubber_asn+"_"+ day +"_"+mon+"_" +year+".csv")
    confirmed_customers1_unique_origin_ases = confirmed_customers1['origin_as'].unique()
    confirmed_customers1_unique_origin_prefixes = confirmed_customers1['prefix'].unique()
    
    path_prepending = pd.read_csv(path + "unique_optimized_provider_as"+scrubber_asn+"_path_prepend_01_"+mon+"_"+year+".csv")
    path_prepending_unique_origin_ases = path_prepending['origin_as'].unique()
    
    path_prepending_unique_origin_prefixes = path_prepending['prefix'].unique()

    confirmed_customers2_ases = list(set(path_prepending_unique_origin_ases)-set(confirmed_customers1_unique_origin_ases))
    confirmed_customers2_prefixes = list(set(path_prepending_unique_origin_prefixes)-set(confirmed_customers1_unique_origin_prefixes))

    df = pd.read_csv(path + "unique_optimized_provider_not_as"+scrubber_asn+"_01_"+ mon +"_"+year+"_v3.csv")
    sibling_path = df.loc[(df['siblings'] == 1) & (df['new_provider_sibling_check'] == int(scrubber_asn))]
    sibling_path_unique_origin_ases = sibling_path['origin_as'].unique()
    sibling_path_unique_origin_prefixes = sibling_path['prefix'].unique()

    confirmed_customers3_ases = list(set(sibling_path_unique_origin_ases)-set(confirmed_customers2_ases)-set(confirmed_customers1_unique_origin_ases))
    confirmed_customers3_prefixes = list(set(sibling_path_unique_origin_prefixes)-set(confirmed_customers2_prefixes)-set(confirmed_customers1_unique_origin_prefixes))

    confirmed_customers_ases = list(confirmed_customers1_unique_origin_ases) + list(confirmed_customers2_ases) + list(confirmed_customers3_ases)
    confirmed_customers_prefixes = list(confirmed_customers1_unique_origin_prefixes) + list(confirmed_customers2_prefixes) + list(confirmed_customers3_prefixes)

#     confirmed_customers_ases.to_csv(path + "final_customers_ases_"+mon+".csv", index = False)
#     confirmed_customers_prefixes.to_csv(path + "final_customers_prefixes_"+mon+".csv", index = False)

    single_record_ases.append(mon)
    single_record_ases.append(year)
    single_record_ases.append(len(confirmed_customers_ases))
    all_record_ases.append(single_record_ases)

    single_record_prefixes.append(mon)
    single_record_prefixes.append(year)
    single_record_prefixes.append(len(confirmed_customers_prefixes))
    all_record_prefixes.append(single_record_prefixes)
    
    
print("Step 7 completed.\n")

In [ ]:
print("Step 8: Saving customer ASes in the csv file.\n")
import csv
headers = ["month", "year", "no_customer_ases"] 

filename_ases = path + "final_confirmed_customer_ases_as" + scrubber_asn + "_"+year+".csv"

with open(filename_ases, 'a') as f:
    write = csv.writer(f)
#     write.writerow(headers)
    write.writerows(all_record_ases)

print("Step 8 completed.")

In [ ]:
print("Step 9: Saving customer prefixes in the csv file.\n")
import csv

# header names 
headers = ["month", "year", "no_customer_prefixes"] 

filename_prefixes = path + "final_confirmed_customer_prefixes_as" + scrubber_asn + "_"+year+".csv"

with open(filename_prefixes, 'a') as f: 
    write = csv.writer(f)
#     write.writerow(headers) # Remove header after the first file is processed.
    write.writerows(all_record_prefixes)

print("Step 9 completed.")

In [ ]:
print("Step 9: Storing files in the DACS object storage.")
import os
os.system("mc mb obj-dacs/catrin/data_processing/tool=ddos_scrubber/customer/as"+ scrubber_asn + "/"+year)
os.system("mc cp -r "+ path +"/* obj-dacs/catrin/data_processing/tool=ddos_scrubber/customer/as"+scrubber_asn + "/"+year)
print("Step 10 completed.")